In [17]:
import numpy as np
import os, sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), 'C:/Users/jaina/Downloads/keras-squeezenet-master'))) # To import keras_squeezenet.
from keras_squeezenet import SqueezeNet
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image

In [18]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm  
import keras
import math
from scipy import ndimage

In [19]:
import keras
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [20]:
tf.config.experimental.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [21]:
train_dir = "C:/Users/jaina/Downloads/hand_data/processed_images/"

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

     
train_datagen = ImageDataGenerator(rescale=1./255,
    
    #data_format='channels_first',
    validation_split=0.2) 

train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size=128,
    target_size=(227,227),
    shuffle = True,
    classes = ['0','1','2','3','4','5','6','7','8','9','+','-','times','forward_slash','(',')','{','}','[',']','e','i','in','pi'],
    class_mode="sparse",
    subset='training') 

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    batch_size=128,
    target_size=(227,227),
    shuffle = True,
    classes = ['0','1','2','3','4','5','6','7','8','9','+','-','times','forward_slash','(',')','{','}','[',']','e','i','in','pi'],
    class_mode="sparse",
    subset='validation') 

Found 158646 images belonging to 24 classes.
Found 39649 images belonging to 24 classes.


In [23]:
squeeze= SqueezeNet(weights='imagenet')

In [24]:
squeeze.summary()

Model: "squeezenet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 227, 227, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 113, 113, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 113, 113, 64) 0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 56, 56, 64)   0           relu_conv1[0][0]                 
_________________________________________________________________________________________

In [25]:
#num_layers_to_pop =1
#for idx in range(num_layers_to_pop):
    #print(squeeze.layers.pop())

In [26]:
squeeze.summary()

Model: "squeezenet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 227, 227, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 113, 113, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 113, 113, 64) 0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 56, 56, 64)   0           relu_conv1[0][0]                 
_________________________________________________________________________________________

In [27]:
from keras.layers import Dense, Activation, Flatten
model=keras.models.Sequential()
model.add(SqueezeNet(include_top=False,pooling='avg'))


In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
squeezenet (Model)           (None, 512)               722496    
Total params: 722,496
Trainable params: 722,496
Non-trainable params: 0
_________________________________________________________________


In [29]:
for layer in model.layers[:1]:
    layer.trainable=False
#model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(24,activation='softmax'))
    
for layer in model.layers:
    print(layer.name,layer.trainable)
    print('\n')

squeezenet False


dense_3 True


dense_4 True




In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
squeezenet (Model)           (None, 512)               722496    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 24)                6168      
Total params: 859,992
Trainable params: 137,496
Non-trainable params: 722,496
_________________________________________________________________


In [31]:
from keras import optimizers
ada = keras.optimizers.Adadelta(learning_rate=1, rho=0.95)
model.compile(optimizer=ada,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit_generator(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=100,
                              validation_steps=100,
                              epochs=95)

Epoch 1/95
100/100 [==============================] - 37s 373ms/step - loss: 2.5469 - accuracy: 0.2759 - val_loss: 2.4323 - val_accuracy: 0.2596
Epoch 2/95
100/100 [==============================] - 37s 370ms/step - loss: 2.0361 - accuracy: 0.4091 - val_loss: 1.7830 - val_accuracy: 0.4503
Epoch 3/95
100/100 [==============================] - 38s 384ms/step - loss: 1.8015 - accuracy: 0.4516 - val_loss: 1.5986 - val_accuracy: 0.4231
Epoch 4/95
100/100 [==============================] - 36s 357ms/step - loss: 1.6067 - accuracy: 0.4934 - val_loss: 1.8235 - val_accuracy: 0.5399
Epoch 5/95
100/100 [==============================] - 35s 349ms/step - loss: 1.4410 - accuracy: 0.5452 - val_loss: 1.4473 - val_accuracy: 0.5360
Epoch 6/95
100/100 [==============================] - 35s 349ms/step - loss: 1.3788 - accuracy: 0.5616 - val_loss: 1.2744 - val_accuracy: 0.5802
Epoch 7/95
100/100 [==============================] - 36s 364ms/step - loss: 1.2692 - accuracy: 0.5991 - val_loss: 1.4118 - val_ac

In [32]:
model.save("C:/Users/jaina/Downloads/hand_data/model.h5")